In [ ]:
# Importing Required Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

train_data.head()

In [ ]:
X_train = train_data.iloc[:, 1:]
y_train=train_data[["label"]]

print("X_train shape: \t{} \ny_train shape: \t{}".format(X_train.shape, y_train.shape))

## Normalizing

In [ ]:
X_train = X_train.values/255.0

## Reshaping

In [ ]:
X_train = X_train.reshape(X_train.shape[0],28,28)
X_train.shape

## Plotting an image

In [ ]:
plt.imshow(X_train[42])
plt.grid(False)
plt.gray()
# plt.axis('off')
plt.show()

## Model Building and Training

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nStopping training with accuracy: {}".format(logs.get('accuracy')))
      self.model.stop_training = True
        
callbacks = myCallback()

In [ ]:
# Reshaping into 4dim before feeding the model
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_train.shape

In [ ]:
print(model.summary)
history = model.fit(X_train, y_train,epochs=100, callbacks=[callbacks])

In [ ]:
metrics = pd.DataFrame(model.history.history)

metrics[['accuracy','loss']].plot(figsize=(10, 5))

## Prediction with test data

In [ ]:
X_test = test_data.copy()
X_test.shape

In [ ]:
# Normalizing
X_test = X_test.values/255.0

# Reshaping
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_test.shape

In [ ]:
# prediction
prediction = model.predict(X_test)

In [ ]:
sample_submisison = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
sample_submisison.head()

In [ ]:
# Rounding
prediction = np.array(np.round(prediction), dtype='int64')

# Converting multiple(10) columns into one
prediction = (np.argmax(prediction, axis=1))

# Creating dataframe
prediction = pd.DataFrame({'ImageId': range(1,len(prediction)+1), 'Label': prediction})

In [ ]:
prediction.head()

In [ ]:
prediction.to_csv('submission.csv',index=False)